# <center> **Оптимизация гиперпараметров. Практика.**

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn import model_selection #сплитование выборки

import warnings
warnings.filterwarnings("ignore") #игнорируем предупреждения, чтобы глаза не мозолили

from hyperopt import hp, fmin, tpe, Trials
import optuna 

plt.style.use('seaborn')

## Данные

In [2]:
data = pd.read_csv('data/_train_sem09__1_.zip')

data

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


## Разделение на выборки

In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y, random_state=1, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3000, 1776), (751, 1776), (3000,), (751,))

### Функция обучения

In [4]:
def auto_learning(model):
    """Чтобы не прописывать одно и то же много раз, 
       я создал функцию, которая сама обучает модель 
       и выводит значения метрик на экран."""
       
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    f1_train = round(metrics.f1_score(y_train, y_train_pred), 2)
    f1_test = round(metrics.f1_score(y_test, y_test_pred), 2)

    print(f'Train F1-score: {f1_train}')
    print(f'Test F1-score: {f1_test}')
    
# random state
R = 10

## Baseline-модели

In [5]:
# Логистическая регрессия 
lr = linear_model.LogisticRegression(random_state=R, max_iter=50)

auto_learning(lr)

Train F1-score: 0.87
Test F1-score: 0.79


In [6]:
# Случайный лес
rf = ensemble.RandomForestClassifier(random_state=R) 

auto_learning(rf)

Train F1-score: 1.0
Test F1-score: 0.82


## Метод `GridSearchCV` 

### Функция отчета

In [7]:
def params_and_score_report(search):
    """Функция для обучения и вывода отчета o модели"""
    
    search.fit(X_train, y_train) 
    
    print(f"Наилучшие значения параметров: {search.best_params_}")
    print(f"f1_score на тестовом наборе: {round(search.score(X_test, y_test), 2)}")

### Сетки параметров

In [8]:
# для логистической регрессии
params_lr = {'penalty': ['l2', None] , 
             'solver': ['lbfgs', 'sag'], 
             'C': list(np.linspace(0.01, 1, 10, dtype=float))} 

# для случайного леса
params_rf = {'n_estimators': [100, 150, 200], 
             'min_samples_leaf': [3, 5], 
             'max_depth': [10, 15, 20]} 

### Оптимизация

In [9]:
# Логистическая регрессия 
grid_search = model_selection.GridSearchCV(estimator=lr, 
                                           param_grid=params_lr, 
                                           cv=5, 
                                           n_jobs=-1, 
                                           scoring='f1')

%time params_and_score_report(grid_search)

Наилучшие значения параметров: {'C': 0.12, 'penalty': 'l2', 'solver': 'sag'}
f1_score на тестовом наборе: 0.79
CPU times: total: 2.12 s
Wall time: 32.8 s


In [10]:
# Случайный лес
grid_search = model_selection.GridSearchCV(estimator=rf, 
                                           param_grid=params_rf, 
                                           cv=5, 
                                           n_jobs=-1, 
                                           scoring='f1')

%time params_and_score_report(grid_search)

Наилучшие значения параметров: {'max_depth': 15, 'min_samples_leaf': 3, 'n_estimators': 150}
f1_score на тестовом наборе: 0.81
CPU times: total: 2.05 s
Wall time: 24.6 s


## Метод `RandomizedSearchCV` 

### Оптимизация (на тех же сетках из предыдущего раздела)

In [11]:
# Логистическая регрессия 
random_search = model_selection.RandomizedSearchCV(estimator=lr, 
                                                   param_distributions=params_lr, 
                                                   cv=5,  
                                                   n_jobs=-1, 
                                                   scoring='f1', 
                                                   n_iter=50)

%time params_and_score_report(random_search)

Наилучшие значения параметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.12}
f1_score на тестовом наборе: 0.79
CPU times: total: 2.06 s
Wall time: 30.6 s


In [12]:
# Случайный лес 
random_search = model_selection.RandomizedSearchCV(estimator=rf, 
                                                   param_distributions=params_rf, 
                                                   cv=5,  
                                                   n_jobs=-1, 
                                                   scoring='f1', 
                                                   n_iter=50)

%time params_and_score_report(random_search)

Наилучшие значения параметров: {'n_estimators': 150, 'min_samples_leaf': 3, 'max_depth': 15}
f1_score на тестовом наборе: 0.81
CPU times: total: 2.11 s
Wall time: 24.6 s


## Метод `HyperOpt` 

### Пространства гиперпараметров

In [13]:
# для логистической регрессии
space_lr = {'penalty': hp.choice('penalty', ['l2', None]), 
            'solver': hp.choice('solver', ['lbfgs', 'sag']), 
            'C': hp.choice('C', list(np.linspace(0.01, 1, 10)))}

# для случайного леса
space_rf = {'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
            'max_depth' : hp.quniform('max_depth', 15, 26, 1),
            'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)} 

### Функции поиска гиперпараметров

In [14]:
def hyperopt_lr(space, cv=5, random_state=R):
    """Функция для поиска гиперпараметров логистической регрессии"""
    
    # функция получает комбинацию гиперпараметров 
    params = {'penalty': space['penalty'], 
              'solver': space['solver'], 
              'C': space['C']}
    
    # модель: логистическая регрессия
    model = linear_model.LogisticRegression(**params, random_state=random_state)
    
    # модель обучается с помощью кросс-валидации
    score = model_selection.cross_val_score(model, 
                                            X_train, 
                                            y_train, 
                                            cv=cv, 
                                            scoring="f1", 
                                            n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score


def hyperopt_rf(space, cv=5, random_state=R):
    """Функция для поиска гиперпараметров случайного леса"""
    
    # функция получает комбинацию гиперпараметров 
    params = {'n_estimators': int(space['n_estimators']), 
              'max_depth': int(space['max_depth']), 
              'min_samples_leaf': int(space['min_samples_leaf'])}
    
    # модель: случайный лес
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    
    # модель обучается с помощью кросс-валидации
    score = model_selection.cross_val_score(model, 
                                            X_train, 
                                            y_train, 
                                            cv=cv, 
                                            scoring="f1", 
                                            n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score 

### Оптимизация

In [15]:
# Логистическая регрессия 
best_lr = fmin(hyperopt_lr, # наша функция 
               space=space_lr, # пространство гиперпараметров
               algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
               max_evals=20, # максимальное количество итераций
               trials=Trials(), # логирование результатов
               rstate=np.random.default_rng(R)) # фиксируем для повторяемости результата

# fmin вернул лишь индексы найденных параметров, исправим их на их значения
for param, value in best_lr.items():
    best_lr[param] = params_lr[param][value]

print(f"Наилучшие значения гиперпараметров {best_lr}")

model = linear_model.LogisticRegression(random_state=R, 
                                        penalty=best_lr['penalty'],
                                        solver=best_lr['solver'],
                                        C=best_lr['C'])

%time auto_learning(model)

100%|██████████| 20/20 [00:53<00:00,  2.70s/trial, best loss: -0.7813491504787413]
Наилучшие значения гиперпараметров {'C': 0.12, 'penalty': 'l2', 'solver': 'sag'}
Train F1-score: 0.85
Test F1-score: 0.79
CPU times: total: 3.12 s
Wall time: 3.16 s


In [16]:
# Случайный лес
best_rf = fmin(hyperopt_rf, # наша функция 
               space=space_rf, # пространство гиперпараметров
               algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
               max_evals=20, # максимальное количество итераций
               trials=Trials(), # логирование результатов
               rstate=np.random.default_rng(R)) # фиксируем для повторяемости результата

print(f"Наилучшие значения гиперпараметров {best_rf}")

model = ensemble.RandomForestClassifier(random_state=R, 
                                        n_estimators=int(best_rf['n_estimators']),
                                        max_depth=int(best_rf['max_depth']),
                                        min_samples_leaf=int(best_rf['min_samples_leaf']))

%time auto_learning(model)

100%|██████████| 20/20 [00:40<00:00,  2.02s/trial, best loss: -0.8175752409410958]
Наилучшие значения гиперпараметров {'max_depth': 16.0, 'min_samples_leaf': 3.0, 'n_estimators': 159.0}
Train F1-score: 0.97
Test F1-score: 0.82
CPU times: total: 2.09 s
Wall time: 2.1 s


## Метод `Optuna` 

### Функции поиска гиперпараметров

In [17]:
def optuna_lr(trial: optuna.trial.Trial):
    """Функция для поиска гиперпараметров логистической регрессии"""
    
    # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l2', None])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sag'])
    c = trial.suggest_float(name='C', low=0.01, high=1, step=10)
    
    # создаем модель
    model = linear_model.LogisticRegression(penalty=penalty, 
                                            solver=solver,
                                            C=c,
                                            random_state=R)
    
    # обучаем модель
    model.fit(X_train, y_train)
    
    return metrics.f1_score(y_train, model.predict(X_train))


def optuna_rf(trial: optuna.trial.Trial):
    """Функция для поиска гиперпараметров случайного леса"""
    
    # задаем пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 100, 300, 10)
    max_depth = trial.suggest_int('max_depth', 15, 40, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)
    
    # создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                            max_depth=max_depth,
                                            min_samples_leaf=min_samples_leaf,
                                            random_state=R)
    
    # обучаем модель
    model.fit(X_train, y_train)
    
    return metrics.f1_score(y_train, model.predict(X_train))

In [18]:
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")

# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params, random_state=R)

%time auto_learning(model)

[I 2023-04-17 13:03:34,579] A new study created in memory with name: LogisticRegression
[I 2023-04-17 13:03:34,901] Trial 0 finished with value: 0.8167067307692307 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.01}. Best is trial 0 with value: 0.8167067307692307.
[I 2023-04-17 13:03:35,317] Trial 1 finished with value: 0.9102053325160895 and parameters: {'penalty': None, 'solver': 'lbfgs', 'C': 0.01}. Best is trial 1 with value: 0.9102053325160895.
[I 2023-04-17 13:03:38,080] Trial 2 finished with value: 0.8170621808350855 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.01}. Best is trial 1 with value: 0.9102053325160895.
[I 2023-04-17 13:03:38,390] Trial 3 finished with value: 0.8167067307692307 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.01}. Best is trial 1 with value: 0.9102053325160895.
[I 2023-04-17 13:03:38,703] Trial 4 finished with value: 0.8167067307692307 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.01}. Best is trial 1 w

Train F1-score: 0.91
Test F1-score: 0.75
CPU times: total: 531 ms
Wall time: 430 ms


In [19]:
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")

# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params, random_state=R)

%time auto_learning(model)

[I 2023-04-17 13:03:55,927] A new study created in memory with name: RandomForestClassifier
[I 2023-04-17 13:03:58,874] Trial 0 finished with value: 0.9321100917431192 and parameters: {'n_estimators': 260, 'max_depth': 40, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9321100917431192.
[I 2023-04-17 13:04:00,484] Trial 1 finished with value: 0.9312557286892759 and parameters: {'n_estimators': 140, 'max_depth': 33, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9321100917431192.
[I 2023-04-17 13:04:03,525] Trial 2 finished with value: 0.9308022045315371 and parameters: {'n_estimators': 270, 'max_depth': 19, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9321100917431192.
[I 2023-04-17 13:04:05,388] Trial 3 finished with value: 0.9216702224931423 and parameters: {'n_estimators': 170, 'max_depth': 23, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9321100917431192.
[I 2023-04-17 13:04:07,752] Trial 4 finished with value: 0.9752369306022622 and parameters: {'n_

Train F1-score: 0.98
Test F1-score: 0.82
CPU times: total: 1.7 s
Wall time: 2.88 s
